# 🦙 LlamaIndex：两个 Chat Engine + 共享 ChatSummaryMemoryBuffer 教程（Feynman 风格）

> 目标：**同时演示 `condense_question` 与 `simple` 两种 chat_engine**，并让它们**共享同一个 `ChatSummaryMemoryBuffer`** 进行多轮对话；**先展示默认的摘要提示词，再演示如何自定义它**。  
> 风格要求：无自定义函数/类、无容错处理；代码输出大量可视化分隔与 emoji，便于观察。

---

## 📚 参考与要点（强烈建议先看）
- Chat Engine 用法与 `condense_question` 说明。
- `SimpleChatEngine`：不检索知识库，只与 LLM 对话；可配合 Memory。  
- `ChatSummaryMemoryBuffer`：在 token 限制下**保留最近消息**并**自动总结更早历史**；支持**自定义总结提示词**。  

---

> ⚙️ 运行前准备：在本地环境中设置你的 OpenAI Key：  
> `setx OPENAI_API_KEY "sk-xxxx"`（Windows）或 `export OPENAI_API_KEY=sk-xxxx`（macOS/Linux）


## 🧩 安装（如需）
如果环境缺少依赖，你可以先运行（**已注释**，按需取消注释）：
```bash
# %pip install -U llama-index llama-index-core llama-index-llms-openai
```


## 1️⃣ 导入 & 全局设置
我们选用 OpenAI 作为 LLM；`Settings` 里统一配置，便于所有组件共享。

In [ ]:
# =============================
# 1) 基础导入与全局设置
# =============================
import os
from llama_index.core import Settings, VectorStoreIndex, PromptTemplate
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.memory import ChatSummaryMemoryBuffer
from llama_index.core.chat_engine import SimpleChatEngine

# 替换成你自己的 OpenAI API Key
os.environ["OPENAI_API_KEY"] = "sk-qY0nr8zudg7Wc2bTR8EUV6rOTwfqZlU2ihwGL4pJ6m2ZEkEE"
# 如需自定义网关（如代理或 Azure），取消下面注释并替换
os.environ["OPENAI_BASE_URL"] = "https://api.openai-proxy.org/v1"



# ➊ 配置全局设置（替代 ServiceContext）
Settings.llm = OpenAI(
    model="gpt-4o-mini", 
    temperature=0.1,
    api_base="https://api.openai-proxy.org/v1"
)
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small",
    api_base="https://api.openai-proxy.org/v1"
)


print("✅ 已设置 LLM 和 Embedding。\n")

✅ 已设置 LLM 和 Embedding。



## 2️⃣ 读取文档并切分为节点（严格按你给的写法）
- `documents = SimpleDirectoryReader('data').load_data()`  
- `splitter = SentenceSplitter(chunk_size=512, chunk_overlap=50)`  
- `nodes = splitter.get_nodes_from_documents(documents)`

In [42]:
# =============================
# 2) 读取 & 切块 --> 节点
# =============================
from llama_index.core import SimpleDirectoryReader

# ⭐ 关键三行（按你的要求保持原样）
documents = SimpleDirectoryReader('../llama_data').load_data()
# 给每个文档加上示例 metadata
for idx, doc in enumerate(documents, start=1):
    doc.metadata["course_id"] = "course_01"  # 真实场景可根据文件夹层级自动填
    doc.metadata["course_material_id"] = f"material_{idx:03d}"
    
documents[0]  # 看看结构

Document(id_='fbe8e638-cc39-4330-a7bc-b5f683cab05d', embedding=None, metadata={'file_path': 'c:\\Users\\Administrator\\Desktop\\ai_mindmap\\proc_history_an_memory\\..\\llama_data\\python第八章.md', 'file_name': 'python第八章.md', 'file_size': 48172, 'creation_date': '2025-08-07', 'last_modified_date': '2025-07-16', 'course_id': 'course_01', 'course_material_id': 'material_001'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='# Python 第八章 函数\n\n在本章中，你将学习编写**函数**（function）。函数是带名字的代码块，用于完成具体的工作。要执行函数定义的特定任务，可**调用**（call）该函数。当需要在程序中多次执行同一项任务时，无须反复编写完成该任务的代码，只需要调用执行该任务的函数，让 Python 运行其中的代码即可。你将发现，使用函数，程序编写、阅读、测试和修复起来都会更容易。\n\n你还将学习各种向函数传递信息的方式，学习编写主要

In [43]:
# 用句子级别的方法切文本块
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=512, chunk_overlap=50)
nodes = splitter.get_nodes_from_documents(documents)

print(f"总共生成了 {len(nodes)} 个节点")
print("=" * 80)

for i, node in enumerate(nodes):
    print(f"节点 {i+1}:")
    print(node.text)
    print(f"字符数量: {len(node.text)}")
    print("=" * 80)

总共生成了 37 个节点
节点 1:
# Python 第八章 函数

在本章中，你将学习编写**函数**（function）。函数是带名字的代码块，用于完成具体的工作。要执行函数定义的特定任务，可**调用**（call）该函数。当需要在程序中多次执行同一项任务时，无须反复编写完成该任务的代码，只需要调用执行该任务的函数，让 Python 运行其中的代码即可。你将发现，使用函数，程序编写、阅读、测试和修复起来都会更容易。

你还将学习各种向函数传递信息的方式，学习编写主要任务是显示信息的函数，以及用于处理数据并返回一个或一组值的函数。最后，你将学习如何将函数存储在称为**模块**（module）的独立文件中，让主程序文件更加整洁。

## 定义函数

下面是一个打印问候语的简单函数，名为 greet_user()：

### greeter.py 示例

```python
def greet_user():
    """ 显示简单的问候语 """
    print("Hello!")
字符数量: 432
节点 2:
greet_user()
```

#### 函数定义的基本结构

这个示例演示了最简单的函数结构。第一行代码使用关键字 def 来告诉 Python，你要定义一个函数。这是**函数定义**，向 Python 指出了函数名，还可以在括号内指出函数为完成任务需要什么样的信息。在这里，函数名为 greet_user()，它不需要任何信息就能完成工作，因此括号内是空的（即便如此，括号也必不可少）。最后，定义以冒号结尾。

#### 函数体和文档字符串

紧跟在 def greet_user():后面的所有缩进行构成了函数体。第二行的文本是称为**文档字符串**（docstring）的注释，描述了函数是做什么的。Python 在为程序中的函数生成文档时，会查找紧跟在函数定义后的字符串。这些字符串通常前后分别用三个双引号引起，能够包含多行。

代码行 print("Hello!")是函数体内的唯一行代码，因此 greet_user()只做一项工作：打印 Hello!。

#### 函数调用

要使用这个函数，必须调用它。**函数调用**让 Python 执行函数中的代码。要调用函数，可依次指定函数名以及用括号括起的必要信息。由于这个函数不需要任何信息，调用它

## 3️⃣ 构建向量索引（供 `condense_question` 模式使用）

In [44]:
# =============================
# 3) 构建 VectorStoreIndex
# =============================
print("\n" + "-"*80)
print("🧱 构建向量索引 VectorStoreIndex ...") 
index = VectorStoreIndex(nodes)
print("✅ 索引就绪！\n")
print("-"*80 + "\n")


--------------------------------------------------------------------------------
🧱 构建向量索引 VectorStoreIndex ...
✅ 索引就绪！

--------------------------------------------------------------------------------



## 4️⃣ 创建一个 **共享** 的 `ChatSummaryMemoryBuffer`
- 该内存将被 **两个 chat_engine** 共同使用  
- 我们先**打印默认的摘要提示词**，再**演示替换为自定义提示词**

In [ ]:
# =============================
# 4) 共享的 ChatSummaryMemoryBuffer
# =============================
print("\n" + "🌟 创建共享 ChatSummaryMemoryBuffer (token_limit=1000) ...\n")

memory = ChatSummaryMemoryBuffer.from_defaults(
    token_limit=,
    llm=Settings.llm  # 用同一个 LLM 进行摘要
)

print(memory.summarize_prompt)


🌟 创建共享 ChatSummaryMemoryBuffer (token_limit=1000) ...

The following is a conversation between the user and assistant. Write a concise summary about the contents of this conversation.


In [ ]:
# —— 自定义摘要提示词 ——
print("✏️ 现在把摘要提示词改成中文、更贴合对于教学材料的学生问答")

custom_summary_prompt = PromptTemplate(
    """你是对话记忆助理。请在 300 字内用总结对话要点，突出：
1) 学生的核心问题和困惑点；
2) 已经给出的关键信息、结论和思路。
"""
)

# 直接替换内存的摘要提示词
memory.summarize_prompt = custom_summary_prompt

print("🎯 已替换为自定义摘要提示词：\n")
print("="*80)
print(memory.summarize_prompt)
print("="*80 + "\n")

✏️ 现在把摘要提示词改成中文、更贴合对于教学材料的学生问答
🎯 已替换为自定义摘要提示词：

metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>} template_vars=[] kwargs={} output_parser=None template_var_mappings=None function_mappings=None template='你是对话记忆助理。请在 100~150 字内用中文总结对话要点，突出：\n1) 学生的核心问题和困惑点；\n2) 已经给出的关键信息与结论；\n3) 下一步可能需要的资料/行动点。\n请尽量使用要点式（•）列出。不要编造未出现的信息。'



## 5️⃣ 建立两个 Chat Engine（共享同一个 Memory）
- **A：`condense_question`**（对历史+本轮进行**问题凝练** ➜ 检索索引 ➜ 答复）  
- **B：`simple`**（**不检索**，直接与 LLM 聊天，但同样利用**共享内存**）

In [47]:
# =============================
# 5) 两个 Chat Engine（共享 memory）
# =============================
print("🚀 A) condense_question 引擎（会用到上面的向量索引）\n")
condense_engine = index.as_chat_engine(
    chat_mode="condense_question",
    memory=memory,
    verbose=True
)

print("💬 B) simple 引擎（不检索，只与LLM聊天，但同样共享 memory）\n")
simple_engine = SimpleChatEngine.from_defaults(
    llm=Settings.llm,
    memory=memory,
    system_prompt=(
        "你是一个友好的对话助手。请简洁、清晰、有条理地回应，并用一点点 emoji。"
    ),
    verbose=True
)
print("✅ 两个引擎均已就绪，且共享同一份 ChatSummaryMemoryBuffer。\n")

🚀 A) condense_question 引擎（会用到上面的向量索引）

💬 B) simple 引擎（不检索，只与LLM聊天，但同样共享 memory）

✅ 两个引擎均已就绪，且共享同一份 ChatSummaryMemoryBuffer。



## 6️⃣ 多轮对话演示：两种引擎**轮流**与用户对话，但**记忆共享**
为了更清晰，我们加入大量分隔线与 emoji。你可以多次运行观察**摘要如何演化**。

In [ ]:
# =============================
# 6) 多轮对话（共享记忆）
# =============================
# 查看 ChatSummaryMemoryBuffer 生成的完整消息列表
from rich.console import Console
from rich.panel import Panel

console = Console(width=120)

def inspect_memory_messages(memory, title="Memory 消息检查"):
    console.print(Panel(f"🔍 {title}", style="bold yellow", width=120))
    
    messages = memory.get()  # 获取最终传给 LLM 的消息
    
    for i, msg in enumerate(messages):
        role = msg.role.value if hasattr(msg.role, 'value') else str(msg.role)
        content = msg.content
        
        style = "red" if role == "system" else "cyan" if role == "user" else "green"
        console.print(f"[{i}] {role.upper()}: {content}", style=style)
        console.print("-" * 80)
        
print("\n" + "#"*90)
print("🎬 场景说明：Python 初学者教程多轮问答，展示共享记忆效果。\n" )

# 1) 用 condense_question（会检索）
print("🔎 [回合1 | condense_question] 用户：'Python 中如何定义函数？'\n")
r1 = condense_engine.chat("Python 中如何定义函数？请给我一个简明扼要的说明和示例。") 
print("🤖 回答：\n", r1, "\n")
inspect_memory_messages(memory, "Round 1 后的消息状态")

# 2) 用 simple（不检索，走闲聊/需求澄清，但仍然写入同一份记忆）
print("🗣️ [回合2 | simple] 用户：'顺便把刚才的要点用 3 条列一下～'\n")
r2 = simple_engine.chat("顺便把你刚才关于函数定义的回答用 3 条要点列一下，简短一点。") 
print("🤖 回答：\n", r2, "\n")
inspect_memory_messages(memory, "Round 2 后的消息状态")

# 3) 再用 condense_question（继续检索式对话，但会带着同一个 memory 的历史）
print("🔎 [回合3 | condense_question] 用户：'函数参数是怎么回事？'\n")
r3 = condense_engine.chat("请基于我们刚才讨论的函数基础，详细解释一下函数参数的概念。") 
print("🤖 回答：\n", r3, "\n")
inspect_memory_messages(memory, "Round 3 后的消息状态")

print("#"*90 + "\n")

# 8修改默认提示词

In [53]:
from llama_index.core.chat_engine.condense_question import DEFAULT_PROMPT

print("=== Condense 默认提示词 ===")
print(DEFAULT_PROMPT.get_template())

=== Condense 默认提示词 ===
Given a conversation (between Human and Assistant) and a follow up message from Human, rewrite the message to be a standalone question that captures all relevant context from the conversation.

<Chat History>
{chat_history}

<Follow Up Message>
{question}

<Standalone question>



In [ ]:
# 修改问题的压缩提示词

from llama_index.core.prompts import PromptTemplate
from llama_index.core.chat_engine import CondenseQuestionChatEngine

new_condense_prompt = PromptTemplate(
    "你是一个RAG（检索增强生成）开发专家，你将根据用户和AI助手之前的{{聊天历史}}，把{{学生最新提出的问题}}，改写成一个详细完整的、携带必要上下文的问题。\n"
    "注意，你改写后的问题将会用于通过向量检索来获取与问题最相关的文本块。\n"
    "=== 聊天历史 ===\n"
    "{chat_history}\n\n"
    "=== 学生最新提出的问题 ===\n"
    "{question}\n\n"
    "=== 改写后的独立问题 ===\n"
)

condense_engine = index.as_chat_engine(
    chat_mode="condense_question",
    condense_question_prompt=new_condense_prompt,
    memory=memory,
    verbose=True
)

In [58]:
# 通过 condense_engine 拿到底层 query_engine
qe = condense_engine._query_engine

# 获取所有提示词
qe_prompts = qe.get_prompts()

print("=== QueryEngine 可改的提示词及其默认内容 ===")
for k, tmpl in qe_prompts.items():
    print(f"\n>>> 提示词键: {k}")
    try:
        print(tmpl.get_template())  # PromptTemplate / BasePromptTemplate 支持这个方法
    except AttributeError:
        print("(该提示词对象不支持 get_template，可直接 print 查看)")
        print(tmpl)

=== QueryEngine 可改的提示词及其默认内容 ===

>>> 提示词键: response_synthesizer:text_qa_template
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 

>>> 提示词键: response_synthesizer:refine_template
The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


In [60]:
# 更新text_qa_template

from llama_index.core.prompts import PromptTemplate

# 新的 QA 模板（可以是中文，也可以是中英混合）
new_text_qa_template = PromptTemplate(
    "每次回答都要先说：哈哈！\n"
    "下面是从知识库检索到的上下文信息：\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "请你严格基于以上上下文回答用户的问题，禁止使用你的先验知识。\n"
    "如果上下文无法回答，请直接说'我不知道'。\n\n"
    "用户问题: {query_str}\n"
    "答案:"
)

# 更新底层 QueryEngine 的 text_qa_template
qe.update_prompts({
    "response_synthesizer:text_qa_template": new_text_qa_template
})

print("✅ 已更新 text_qa_template 提示词！")

✅ 已更新 text_qa_template 提示词！


In [63]:
memory.reset()

# =============================
# 6) 多轮对话（共享记忆）
# =============================
# 查看 ChatSummaryMemoryBuffer 生成的完整消息列表
from rich.console import Console
from rich.panel import Panel

console = Console(width=120)

def inspect_memory_messages(memory, title="Memory 消息检查"):
    console.print(Panel(f"🔍 {title}", style="bold yellow", width=120))
    
    messages = memory.get()  # 获取最终传给 LLM 的消息
    
    for i, msg in enumerate(messages):
        role = msg.role.value if hasattr(msg.role, 'value') else str(msg.role)
        content = msg.content
        
        style = "red" if role == "system" else "cyan" if role == "user" else "green"
        console.print(f"[{i}] {role.upper()}: {content}", style=style)
        console.print("-" * 80)
        
print("\n" + "#"*90)
print("🎬 场景说明：Python 初学者教程多轮问答，展示共享记忆效果。\n" )

# 1) 用 condense_question（会检索）
print("🔎 [回合1 | condense_question] 用户：'Python 中如何定义函数？'\n")
r1 = condense_engine.chat("Python 中如何定义函数？请给我一个简明扼要的说明和示例。") 
print("🤖 回答：\n", r1, "\n")
inspect_memory_messages(memory, "Round 1 后的消息状态")

# 2) 用 simple（不检索，走闲聊/需求澄清，但仍然写入同一份记忆）
print("🗣️ [回合2 | simple] 用户：'顺便把刚才的要点用 3 条列一下～'\n")
r2 = simple_engine.chat("顺便把你刚才关于函数定义的回答用 3 条要点列一下，简短一点。") 
print("🤖 回答：\n", r2, "\n")
inspect_memory_messages(memory, "Round 2 后的消息状态")

# 3) 再用 condense_question（继续检索式对话，但会带着同一个 memory 的历史）
print("🔎 [回合3 | condense_question] 用户：'函数参数是怎么回事？'\n")
r3 = condense_engine.chat("请基于我们刚才讨论的函数基础，详细解释一下函数参数的概念。") 
print("🤖 回答：\n", r3, "\n")
inspect_memory_messages(memory, "Round 3 后的消息状态")

print("#"*90 + "\n")


##########################################################################################
🎬 场景说明：Python 初学者教程多轮问答，展示共享记忆效果。

🔎 [回合1 | condense_question] 用户：'Python 中如何定义函数？'

Querying with: Python 中如何定义函数？请给我一个简明扼要的说明和示例。
🤖 回答：
 哈哈！在 Python 中，定义函数的基本语法如下：

```python
def function_name(parameters):
    """函数的文档字符串，描述函数的功能。"""
    # 函数体
    return value  # 可选的返回值
```

### 示例

下面是一个简单的函数示例，它接受两个参数并返回它们的和：

```python
def add_numbers(a, b):
    """返回两个数字的和。"""
    return a + b
```

在这个示例中，`add_numbers` 是函数名，`a` 和 `b` 是参数，函数体中返回了这两个参数的和。 



╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔍 Round 1 后的消息状态                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

[0] USER: Python 中如何定义函数？请给我一个简明扼要的说明和示例。

--------------------------------------------------------------------------------

[1] ASSISTANT: 哈哈！在 Python 中，定义函数的基本语法如下：

```python
def function_name(parameters):
    """函数的文档字符串，描述函数的功能。"""
    # 函数体
    return value  # 可选的返回值
```

### 示例

下面是一个简单的函数示例，它接受两个参数并返回它们的和：

```python
def add_numbers(a, b):
    """返回两个数字的和。"""
    return a + b
```

在这个示例中，`add_numbers` 是函数名，`a` 和 `b` 是参数，函数体中返回了这两个参数的和。

...

🔎 [回合3 | condense_question] 用户：'函数参数是怎么回事？'

Querying with: 请详细解释一下函数参数的概念，包括参数的类型和作用。
🤖 回答：
 哈哈！函数参数是函数定义中用于接收外部信息的变量。根据上下文，参数可以分为两种类型：形参（parameter）和实参（argument）。

1. **形参**：在函数定义中声明的变量，用于接收调用函数时传递的信息。例如，在函数 `greet_user()` 的定义中，`username` 是一个形参，它代表了函数完成工作所需的信息。

2. **实参**：在调用函数时传递给函数的具体值。例如，在代码 `greet_user('jesse')` 中，`'jesse'` 是一个实参，它被传递给函数 `greet_user()`，并赋值给形参 `username`。

### 参数的类型

- **位置实参**：实参的顺序与形参的顺序相同，Python 根据位置将实参与形参关联。

- **关键字实参**：实参由变量名和值组成，可以不按照顺序传递，Python 根据变量名将实参与形参关联。

- **默认值**：形参可以设置默认值，如果在调用函数时没有提供实参，Python 将使用默认值。

### 参数的作用

参数的主要作用是使函数能够接收外部信息，从而执行特定的操作。通过使用不同类型的参数，函数可以灵活地处理各种输入，增强代码的可重用性和可读性。

总之，理解形参和实参的概念及其类型，有助于更好地使用和定义函数。 



╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔍 Round 3 后的消息状态                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

[0] SYSTEM: 
函数参数是函数定义中用于接收外部信息的变量，理解它们的概念对于编写灵活和可重用的代码至关重要。以下是对函数参数的详细解释
：

### 1. 形参与实参
- **形参（Parameter）**：在函数定义中声明的变量，用于接收调用函数时传递的信息。例如，在函数定义 `def 
greet_user(username):` 中，`username` 是形参。
  
- **实参（Argument）**：在调用函数时传递给函数的具体值。例如，在调用 `greet_user('jesse')` 时，`'jesse'` 
是实参，它被传递给形参 `username`。

### 2. 参数的类型
- **位置实参**：实参的顺序与形参的顺序相同，Python 根据位置将实参与形参关联。例如，`def add(a, b):`，调用时 `add(2, 
3)`，`2` 赋值给 `a`，`3` 赋值给 `b`。

- **关键字实参**：实参由变量名和值组成，可以不按照顺序传递。例如，`add(b=3, a=2)`，Python 根据变量名将实参与形参关联。

- **默认值**：形参可以设置默认值，如果在调用函数时没有提供实参，Python 将使用默认值。例如，`def 
greet(name='Guest'):`，如果调用 `greet()`，则 `name` 将使用默认值 `'Guest'`。

### 3. 参数的作用
参数的主要作用是使函数能够接收外部信息，从而执行特定的操作。通过使用不同类型的参数，函数可以灵活地处理各种输入，增强代码
的可重用性和可读性。

### 总结
理解形参和实参的概念及其类型，有助于更好地使用和定义函数。通过合理使用参数，您可以编写出更灵活和高效的代码。

--------------------------------------------------------------------------------

##########################################################################################



In [ ]:
# =============================
# Round 5: Markdown 文档总结
# =============================
console.print(Panel("🔎 [Round 5 - condense_question] 写一个md文档总结今天我们聊过的内容", style="bold blue", width=120))
r5 = condense_engine.chat("写一个md文档总结今天我们聊过的内容")
console.print(Panel(f"🤖 condense_question 回答：\n{r5}", style="green", width=120))

print_memory_details_rich(memory, "Round 5 - Markdown总结")

---

## ✅ 关键点小结
- `condense_question`：每轮先**把“历史 + 本轮”压缩成独立问题**，再用它去检索知识库，适合**追问**与**上下文不完整**的场景。  
- `simple`：**不检索**，但仍**写入同一份内存**；适合**寒暄/口吻/格式**等不依赖外部知识的交流。  
- `ChatSummaryMemoryBuffer`：在 token 限制下自动把更早的历史**总结进一段摘要**，既控成本又保关键信息。  
- **默认摘要提示词可读可改**：本例演示了 `memory.summary_prompt` 读取与替换。

---

## 📎 附：常见问题
- 如果你需要**持久化内存**，可以结合 *Chat Stores*（本例未覆盖）。
- 如果你想**进一步定制各处 Prompt**，可参考官方的 `get_prompts` / `update_prompts` 用法。

祝你玩得开心！🚀
